# Macronutrient Sugarscape Model

Describe the project here.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns

from IPython.display import HTML
from matplotlib.animation import FuncAnimation

# constants
GRID_SIZE = 50
MAX_PROTEIN = 5
MAX_CARBS = 10
MAX_FAT = 3
N_AGENTS = 50
N_STEPS = 500

Explain the Agent class here.

In [23]:
class Agent:
    def __init__(self, x, y):
        self.x = x
        self.y = y

        self.protein = random.uniform(5, 10)
        self.carbs = random.uniform(5, 10)
        self.fats = random.uniform(2, 5)

        self.strength = 1 + self.protein / 10  # carry capacity
        self.metabolism = max(0.1, 0.5 - self.fats / 20)
        
    def move_energy_and_burn(self):
        """Determine move range based on available macronutrients and burn energy."""
        if self.carbs > 0.1:
            self.carbs -= 0.1
            return 2  # more energy with carbs
        elif self.fats > 0.1:
            self.fats -= 0.1
            return 1  # limited energy with fats
        else:
            # dies of starvation
            self.protein = 0
            self.carbs = 0
            self.fats = 0
            return 0

    def get_priority(self, cell_resources):
        if self.protein < 3:
            return cell_resources[0]  # protein
        if self.carbs < 3:
            return cell_resources[1]  # carbs
        else:
            return cell_resources[2]  # fats

    def is_alive(self):
        return (self.protein + self.carbs + self.fats) > 0

Explain the Sugarscape class here.

In [24]:
class Sugarscape:
    def __init__(self, num_agents=50):
        self.grid = np.zeros((GRID_SIZE, GRID_SIZE, 3))  # protein, carbs, fats
        self.agents = [Agent(
            random.randint(0, GRID_SIZE-1),
            random.randint(0, GRID_SIZE-1)
        ) for _ in range(num_agents)]
        self.initialize_resources()

        # set up figure
        sns.set_style("whitegrid")
        self.fig, self.ax = plt.subplots(figsize=(10, 8))

        # initialize visualization elements
        self.img = self.ax.imshow(np.zeros((GRID_SIZE, GRID_SIZE, 3)))
        self.scatter = self.ax.scatter([], [], color='white', s=50, edgecolor='black', linewidth=0.5)
        self.title = self.ax.set_title("Initializing...")
        self.ax.axis('off')
        plt.close(self.fig)  # prevents duplicate display

    def initialize_resources(self):
        for _ in range(1000):
            x, y = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
            self.grid[x, y, 0] = random.uniform(0, MAX_PROTEIN / 2)
            self.grid[x, y, 1] = random.uniform(0, MAX_CARBS / 2)
            self.grid[x, y, 2] = random.uniform(0, MAX_FATS / 2)
    
    def regenerate_resources(self):
        for _ in range(100):
            x, y = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
            self.grid[x, y, 0] = np.minimum(self.grid[x, y, 0] + 0.1, MAX_PROTEIN)
            self.grid[x, y, 1] = np.minimum(self.grid[x, y, 1] + 0.15, MAX_CARBS)
            self.grid[x, y, 2] = np.minimum(self.grid[x, y, 2] + 0.05, MAX_FATS)

    def update(self, frame):
        self.regenerate_resources()
        
        # move agents
        for agent in [a for a in self.agents if a.is_alive()]:
            best_priority = -1
            best_pos = (agent.x, agent.y)
            move_range = agent.move_energy_and_burn()
            if move_range == 0:
                continue  # skip dead agent

            for dx in range(-move_range, move_range + 1):

                for dy in range(-move_range, move_range + 1):

                    nx, ny = agent.x + dx, agent.y + dy
                    if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                        priority = agent.get_priority(self.grid[nx, ny, :])
                        if priority > best_priority:
                            best_priority = priority
                            best_pos = (nx, ny)
            agent.x, agent.y = best_pos
        
        # consume resources
        for agent in [a for a in self.agents if a.is_alive()]:
            p, c, f = self.grid[agent.x, agent.y, :]
            take_p = min(p, agent.strength)
            take_c = min(c, agent.strength)
            take_f = min(f, agent.strength)
            
            self.grid[agent.x, agent.y, 0] -= take_p
            self.grid[agent.x, agent.y, 1] -= take_c
            self.grid[agent.x, agent.y, 2] -= take_f
            
            agent.protein += take_p
            agent.carbs += take_c
            agent.fats += take_f
            
            agent.protein = max(0, agent.protein - agent.metabolism)
            agent.carbs = max(0, agent.carbs - agent.metabolism)
            agent.fats = max(0, agent.fats - agent.metabolism * 0.5)
        
        # update visualization
        rgb_grid = np.zeros((GRID_SIZE, GRID_SIZE, 3))
        rgb_grid[:, :, 0] = self.grid[:, :, 0] / MAX_PROTEIN
        rgb_grid[:, :, 1] = self.grid[:, :, 1] / MAX_CARBS
        rgb_grid[:, :, 2] = self.grid[:, :, 2] / MAX_FATS
        
        self.img.set_array(np.clip(rgb_grid, 0, 1))
        
        alive_agents = [a for a in self.agents if a.is_alive()]
        if alive_agents:
            self.scatter.set_offsets(np.column_stack((
                [a.y for a in alive_agents],
                [a.x for a in alive_agents]
            )))
        else:
            self.scatter.set_offsets(np.empty((0, 2)))
        
        self.title.set_text(
            f"Step {frame}/{N_STEPS} | "
            f"Agents Alive: {len(alive_agents)}\n"
            f"Protein (Red) | Carbs (Green) | Fats (Blue)"
        )
        
        return self.img, self.scatter, self.title

    def run(self):
        self.ani = FuncAnimation(
            self.fig, 
            self.update, 
            frames=N_STEPS,
            interval=50, 
            blit=True
        )
        return HTML(self.ani.to_jshtml())

Run this block to view the simulation:

In [25]:
sim = Sugarscape()
sim.run()

NameError: name 'generate_initial_macros' is not defined